In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


shop_zipcode = '54901' #shop's zipcode for relative monitoring

In [86]:
#importing the data tables

df = pd.read_csv("Customer_Final_Purchasing_Dataset.csv")
products = pd.read_csv("Products_Description.csv")


In [87]:
df.info()
products.info()
#as we can see, no null values in any table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 10 columns):
TransactionID          440 non-null object
Transaction            440 non-null object
CustomerID             440 non-null object
Mobile_number          440 non-null int64
Customer Name          440 non-null object
Age                    440 non-null int64
Billing Without GST    440 non-null int64
Billing With GST       440 non-null float64
Date                   440 non-null object
Zip Code               440 non-null int64
dtypes: float64(1), int64(4), object(5)
memory usage: 34.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 4 columns):
Product Name     13 non-null object
Costing Price    13 non-null float64
Selling Price    13 non-null int64
Profit           13 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 544.0+ bytes


In [88]:
#looking into the data
df

,TransactionID,Transaction,CustomerID,Mobile_number,Customer Name,Age,Billing Without GST,Billing With GST,Date,Zip Code
0,TI0001,"['Camera', 'Camera-holder', 'SD-card']",CUST023,7023711077,Safyr,28,53750,60200.0,2017-01-01,54935
1,TI0002,"['Camera', 'Focus-lens', 'Camera-holder', 'SD-...",CUST012,7023711088,Ballinamore,24,68750,77000.0,2017-01-01,54169
2,TI0003,"['Auto-focus-mount-adapter', 'Camera', 'Photog...",CUST034,7023711066,Sahib,59,70050,78456.0,2017-01-01,54964
3,TI0004,"['Camera', 'Focus-lens', 'Camera-holder', 'SD-...",CUST008,7023711092,Ballad,12,68750,77000.0,2017-01-01,54061
4,TI0005,"['Camera', 'Photographic-lighting', 'Camera-ho...",CUST029,7023711071,Sagiv,30,56100,62832.0,2017-01-01,54952
...,...,...,...,...,...,...,...,...,...,...
435,TI0436,"['Screen-Protector', 'Cover-Case']",CUST006,7023711094,Aaran,11,1400,1568.0,2017-06-21,53919
436,TI0437,"['Sim', 'Power-Bank', 'SD-card']",CUST034,7023711066,Sahib,59,3900,4368.0,2017-06-22,54964
437,TI0438,"['Screen-Protector', 'Sim', 'SD-card']",CUST021,7023711079,Safak,27,1700,1904.0,2017-06-23,54927
438,TI0439,"['Mobile-phone', 'Sim']",CUST010,7023711090,Ballentine,23,45150,50568.0,2017-06-24,54136


In [89]:
df["CustomerID"].value_counts()

CUST038    17
CUST021    16
CUST007    16
CUST015    16
CUST031    15
CUST016    14
CUST033    14
CUST037    14
CUST024    13
CUST032    13
CUST011    13
CUST039    13
CUST005    12
CUST029    12
CUST025    12
CUST017    12
CUST019    12
CUST001    11
CUST036    11
CUST034    11
CUST040    11
CUST012    10
CUST009    10
CUST028    10
CUST027    10
CUST023    10
CUST002     9
CUST008     9
CUST004     9
CUST022     9
CUST020     8
CUST003     8
CUST010     8
CUST014     8
CUST018     8
CUST030     8
CUST006     8
CUST013     7
CUST026     7
CUST035     6
Name: CustomerID, dtype: int64

In [90]:
#now as we can, we need to first check for the outliers in our data

#for now we can see that a person living far away is not going to come to the shop and thus is not our target audience
#and thus is outlier or noise

#So for finding the outlier, we will find the distance on the basis of zipcode of the customer address
#with shop at centre with zipcode = '54901'

#distance formula, d(in miles) = 3956.0 * arccos[(sin(lat1) * sin(lat2)) + cos(lat1) * cos(lat2) * cos(long2 – long1)]

from math import radians, cos, sin, asin, sqrt 
def distance(zipcode):
    lat1 = zcdb[zipcode].latitude
    lat2 = zcdb[shop_zipcode].latitude
    lon1 = zcdb[zipcode].longitude
    lon2 = zcdb[shop_zipcode].longitude
    
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
    lat1 = radians(lat1) 
    lat2 = radians(lat2)
    
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    r = 3956.0
    return (r*c)

possible_zipcode = df["Zip Code"].unique()
distances = [distance(i) for i in possible_zipcode]
#distancemapping = {}
#for i in range(len(distances)):
#    distancemapping[possible_zipcode[i]] = distances[i]

distances
    
#zcdb[zipcodes[0]].latitude

[19.6616390008696,
 15.6906772006554,
 12.645269836218123,
 25.90386100782592,
 13.19458054174663,
 9.751528151845916,
 241.2100892548717,
 26.513258805281556,
 7.1396298086003,
 18.841645339551047,
 26.038100799820807,
 17.249988864310872,
 20.95897899023877,
 11.53169386078454,
 20.37062465184263,
 18.14602146183728,
 18.01510871811929,
 12.413161157365934,
 19.038173568007526,
 12.849959544959619,
 20.129113746678875,
 284.3477636728283,
 11.28742430464468,
 24.494119117110653,
 5.696990876525537,
 5.696990876525537,
 15.395384358609832,
 16.368147194326532,
 17.5349017431494,
 11.818581681907544,
 24.73599289532506,
 12.691328475205337,
 21.26503848867789,
 19.62270983461613,
 3.8668600984860864,
 4.240673638574094,
 18.35654146359636,
 15.906429254679377,
 20.306504721836443,
 16.15728094703058]

In [91]:
#as we can see maximum of the customers live in a radius of 30 miles, only 2 live outside of it
#so we will remove these rows

distance_far_away = []
for i in distances:
    if i>30:
        distance_far_away.append(i)
Zipcode_of_faraway_distance = []
for i in distance_far_away:
    Zipcode_of_faraway_distance.append(possible_zipcode[distances.index(i)])
Zipcode_of_faraway_distance

for i in Zipcode_of_faraway_distance:
    df.drop(df[df["Zip Code"] == i].index,inplace = True)


df

,TransactionID,Transaction,CustomerID,Mobile_number,Customer Name,Age,Billing Without GST,Billing With GST,Date,Zip Code
0,TI0001,"['Camera', 'Camera-holder', 'SD-card']",CUST023,7023711077,Safyr,28,53750,60200.0,2017-01-01,54935
1,TI0002,"['Camera', 'Focus-lens', 'Camera-holder', 'SD-...",CUST012,7023711088,Ballinamore,24,68750,77000.0,2017-01-01,54169
2,TI0003,"['Auto-focus-mount-adapter', 'Camera', 'Photog...",CUST034,7023711066,Sahib,59,70050,78456.0,2017-01-01,54964
3,TI0004,"['Camera', 'Focus-lens', 'Camera-holder', 'SD-...",CUST008,7023711092,Ballad,12,68750,77000.0,2017-01-01,54061
4,TI0005,"['Camera', 'Photographic-lighting', 'Camera-ho...",CUST029,7023711071,Sagiv,30,56100,62832.0,2017-01-01,54952
...,...,...,...,...,...,...,...,...,...,...
435,TI0436,"['Screen-Protector', 'Cover-Case']",CUST006,7023711094,Aaran,11,1400,1568.0,2017-06-21,53919
436,TI0437,"['Sim', 'Power-Bank', 'SD-card']",CUST034,7023711066,Sahib,59,3900,4368.0,2017-06-22,54964
437,TI0438,"['Screen-Protector', 'Sim', 'SD-card']",CUST021,7023711079,Safak,27,1700,1904.0,2017-06-23,54927
438,TI0439,"['Mobile-phone', 'Sim']",CUST010,7023711090,Ballentine,23,45150,50568.0,2017-06-24,54136


In [92]:
#as we can see above, for apriori algorithm their are many unwanted column in our data, so we will drop them
list1 = ["CustomerID", "Mobile_number", "Customer Name", "Age" ,"Billing Without GST", "Billing With GST", "Date","Zip Code"]
df_apriori = df.drop(list1,axis=1)

def Camera(row):
    row = row.split("'")
    if "Camera" in row:
        return 1
    else:
        return 0

def Mobile_phone(row):
    row = row.split("'")
    if "Mobile-phone" in row:
        return 1
    else:
        return 0
    
def Screen_Protector(row):
    row = row.split("'")
    if "Screen-Protector" in row:
        return 1
    else:
        return 0

def Sim(row):
    row = row.split("'")
    if "Sim" in row:
        return 1
    else:
        return 0
    
def Cover_Case(row):
    row = row.split("'")
    if "Cover-Case" in row:
        return 1
    else:
        return 0
    
def Earphones(row):
    row = row.split("'")
    if "Earphones" in row:
        return 1
    else:
        return 0
    
def Power_Bank(row):
    row = row.split("'")
    if "Power-Bank" in row:
        return 1
    else:
        return 0
    
def Focus_lens(row):
    row = row.split("'")
    if "Focus-lens" in row:
        return 1
    else:
        return 0
    
def SD_card(row):
    row = row.split("'")
    if "SD-card" in row:
        return 1
    else:
        return 0
    
def Camera_holder(row):
    row = row.split("'")
    if "Camera-holder" in row:
        return 1
    else:
        return 0
    
def Auto_focus_mount_adapter(row):
    row = row.split("'")
    if "Auto-focus-mount-adapter" in row:
        return 1
    else:
        return 0
    
def Tripod_stand(row):
    row = row.split("'")
    if "Tripod-stand" in row:
        return 1
    else:
        return 0
    
def Photographic_lighting(row):
    row = row.split("'")
    if "Photographic-lighting" in row:
        return 1
    else:
        return 0
    
    
    
df_apriori["Camera"] = df_apriori["Transaction"].apply(Camera)
df_apriori["Mobile Phone"] = df_apriori["Transaction"].apply(Mobile_phone)
df_apriori["Screen Protector"] = df_apriori["Transaction"].apply(Screen_Protector)
df_apriori["Sim"] = df_apriori["Transaction"].apply(Sim)
df_apriori["Cover Case"] = df_apriori["Transaction"].apply(Cover_Case)
df_apriori["Earphones"] = df_apriori["Transaction"].apply(Earphones)
df_apriori["Power Bank"] = df_apriori["Transaction"].apply(Power_Bank)
df_apriori["Focus lens"] = df_apriori["Transaction"].apply(Focus_lens)
df_apriori["SD-card"] = df_apriori["Transaction"].apply(SD_card)
df_apriori["Camera Holder"] = df_apriori["Transaction"].apply(Camera_holder)
df_apriori["Auto Focus Mount Adapter"] = df_apriori["Transaction"].apply(Auto_focus_mount_adapter)
df_apriori["Tripod Stand"] = df_apriori["Transaction"].apply(Tripod_stand)
df_apriori["Photographic Lighting"] = df_apriori["Transaction"].apply(Photographic_lighting)
    
    
    
#now we don't require the transaction attribute too, so we will drop it and make TransactionID as index of dataframe 
df_apriori.drop(["Transaction"],axis=1,inplace=True)
df_apriori.set_index("TransactionID",inplace = True)
df_apriori

,Camera,Mobile Phone,Screen Protector,Sim,Cover Case,Earphones,Power Bank,Focus lens,SD-card,Camera Holder,Auto Focus Mount Adapter,Tripod Stand,Photographic Lighting
TransactionID,,,,,,,,,,,,,
TI0001,1,0,0,0,0,0,0,0,1,1,0,0,0
TI0002,1,0,0,0,0,0,0,1,1,1,0,0,0
TI0003,1,0,0,0,0,0,0,0,0,0,1,1,1
TI0004,1,0,0,0,0,0,0,1,1,1,0,0,0
TI0005,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TI0436,0,0,1,0,1,0,0,0,0,0,0,0,0
TI0437,0,0,0,1,0,0,1,0,1,0,0,0,0
TI0438,0,0,1,1,0,0,0,0,1,0,0,0,0


In [93]:
#products has nothing preprocessing part as it will be only used for fiding the profit per productand overall profit
products

,Product Name,Costing Price,Selling Price,Profit
0,Mobile-phone,40500.0,45000,4500.0
1,Screen-Protector,90.0,300,210.0
2,Sim,120.0,150,30.0
3,Cover-Case,440.0,1100,660.0
4,Earphones,2000.0,2500,500.0
5,Power-Bank,1250.0,2500,1250.0
6,Camera,35000.0,50000,15000.0
7,Focus-lens,9600.0,15000,5400.0
8,SD-card,1187.5,1250,62.5
9,Camera-holder,1875.0,2500,625.0


In [94]:
#Training the model using apriori
my_frequent_itemsets = apriori(df_apriori , min_support = 0.1, use_colnames = True)
my_rules = association_rules(my_frequent_itemsets, metric='lift', min_threshold=1)
my_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Camera),(Focus lens),0.353365,0.182692,0.120192,0.340136,1.861797,0.055635,1.238600
1,(Focus lens),(Camera),0.182692,0.353365,0.120192,0.657895,1.861797,0.055635,1.890163
2,(Camera),(SD-card),0.353365,0.471154,0.211538,0.598639,1.270582,0.045049,1.317634
3,(SD-card),(Camera),0.471154,0.353365,0.211538,0.448980,1.270582,0.045049,1.173522
4,(Camera),(Camera Holder),0.353365,0.300481,0.197115,0.557823,1.856435,0.090936,1.581990
5,(Camera Holder),(Camera),0.300481,0.353365,0.197115,0.656000,1.856435,0.090936,1.879752
6,(Mobile Phone),(Sim),0.266827,0.254808,0.112981,0.423423,1.661737,0.044991,1.292443
7,(Sim),(Mobile Phone),0.254808,0.266827,0.112981,0.443396,1.661737,0.044991,1.317226
8,(SD-card),(Focus lens),0.471154,0.182692,0.100962,0.214286,1.172932,0.014885,1.040210
9,(Focus lens),(SD-card),0.182692,0.471154,0.100962,0.552632,1.172932,0.014885,1.182127


In [95]:
my_rules[ (my_rules["lift"]>=1.5) & (my_rules["confidence"]>=0.5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(Focus lens),(Camera),0.182692,0.353365,0.120192,0.657895,1.861797,0.055635,1.890163
4,(Camera),(Camera Holder),0.353365,0.300481,0.197115,0.557823,1.856435,0.090936,1.581990
5,(Camera Holder),(Camera),0.300481,0.353365,0.197115,0.656000,1.856435,0.090936,1.879752
11,(Focus lens),(Camera Holder),0.182692,0.300481,0.100962,0.552632,1.839158,0.046066,1.563631
14,"(Camera, SD-card)",(Camera Holder),0.211538,0.300481,0.112981,0.534091,1.777455,0.049418,1.501407
16,"(SD-card, Camera Holder)",(Camera),0.170673,0.353365,0.112981,0.661972,1.873335,0.052671,1.912961


In [96]:
df_apriori[df_apriori["Camera"]==1]
#df_apriori[df_apriori["Camera Holder"]==1]


,Camera,Mobile Phone,Screen Protector,Sim,Cover Case,Earphones,Power Bank,Focus lens,SD-card,Camera Holder,Auto Focus Mount Adapter,Tripod Stand,Photographic Lighting
TransactionID,,,,,,,,,,,,,
TI0001,1,0,0,0,0,0,0,0,1,1,0,0,0
TI0002,1,0,0,0,0,0,0,1,1,1,0,0,0
TI0003,1,0,0,0,0,0,0,0,0,0,1,1,1
TI0004,1,0,0,0,0,0,0,1,1,1,0,0,0
TI0005,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TI0216,1,0,0,0,0,0,0,0,0,1,1,0,0
TI0217,1,0,0,0,0,0,0,0,1,1,0,0,0
TI0218,1,0,0,0,0,0,0,1,1,0,0,0,0


In [97]:
#Now as we have find the frequent patterns, let's check the individual product
profit = []
templist = list(products["Product Name"])
templist1 = list(df_apriori.columns)
for i in range(7):
    profit.append(len(df_apriori[df_apriori[templist1[i+1]] == 1])*int(products.loc[products["Product Name"] == templist[i]]["Profit"]))
profit.append( len(df_apriori[df_apriori["Camera"] == 1])*int(products.loc[products["Product Name"] == "Camera"]["Profit"]))
for i in range(8,13):
    profit.append(len(df_apriori[df_apriori[templist1[i]] == 1])*int(products.loc[products["Product Name"] == templist[i]]["Profit"]))
    
#for i in products["Product Name"]:
#    if i == "Mobile-phone"
#    profit.append(len(df_apriori[df_apriori[i] == 1])*(products.loc[products["Product Name"] == i]["Profit"]))
products["Total Profit"] = profit
products.sort_values(by = ["Total Profit"])

,Product Name,Costing Price,Selling Price,Profit,Total Profit
2,Sim,120.0,150,30.0,3180
8,SD-card,1187.5,1250,62.5,12152
1,Screen-Protector,90.0,300,210.0,17220
11,Tripod-stand,1172.5,1750,577.5,17887
5,Power-Bank,1250.0,2500,1250.0,18750
12,Photographic-lighting,2700.0,3600,900.0,26100
4,Earphones,2000.0,2500,500.0,27000
3,Cover-Case,440.0,1100,660.0,31680
9,Camera-holder,1875.0,2500,625.0,78125
10,Auto-focus-mount-adapter,12495.0,14700,2205.0,83790


In [73]:
#as we can we got most of profit from selling the focus-lens, camera and mobile-phone, and other profit was good but
#with very low profit in Sim.

['Camera',
 'Mobile Phone',
 'Screen Protector',
 'Sim',
 'Cover Case',
 'Earphones',
 'Power Bank',
 'Focus lens',
 'SD-card',
 'Camera Holder',
 'Auto Focus Mount Adapter',
 'Tripod Stand',
 'Photographic Lighting']

In [3]:
temp = df["Transaction"][0]
temp

"['Camera', 'SD-card', 'Camera-holder']"

In [4]:
temp.split("'")

['[', 'Camera', ', ', 'SD-card', ', ', 'Camera-holder', ']']

In [9]:
from pyzipcode import ZipCodeDatabase
zcdb = ZipCodeDatabase()
zipcode = zcdb[54115]
print (zipcode.zip)
print (zipcode.city)
print (zipcode.state)
print (zipcode.longitude)
print (zipcode.latitude)
print (zipcode.timezone)

54115
De Pere
WI
-88.07896
44.42042
-6


In [35]:
from pyzipcode import ZipCodeDatabase
zcdb = ZipCodeDatabase()
centre = '54901'
zipcodes = [z.zip for z in zcdb.get_zipcodes_around_radius('54901', 18)]
zipcodes = zipcodes[0:39]
zipcodes.append('55404')
zipcodes.append('50201')
zipcodes.remove('54901')
print(zcdb[zipcodes[0]].latitude)
print(zcdb[zipcodes[0]].longitude)

43.76983
-88.494357


In [30]:
zipcodes

['53009',
 '53019',
 '53049',
 '53065',
 '53088',
 '53919',
 '54035',
 '54061',
 '54113',
 '54136',
 '54140',
 '54169',
 '54902',
 '54903',
 '54904',
 '54906',
 '54911',
 '54913',
 '54914',
 '54915',
 '54927',
 '54932',
 '54935',
 '54936',
 '54937',
 '54942',
 '54944',
 '54947',
 '54952',
 '54956',
 '54957',
 '54961',
 '54963',
 '54964',
 '54969',
 '54974',
 '54979',
 '54980',
 '55404',
 '50201']

In [14]:
temp = zcdb.find_zip(state = "Chennai")

In [15]:
temp[0]

TypeError: 'NoneType' object is not subscriptable